# Here is the Notebook for looking at the LR attacker results

Let's start by looking at all the data

In [8]:
import numpy as np 
import results
from IPython.display import display

rg = results.resultGatherer()
df = rg.gatherResults()
display(df.sort_values(by=['s.mat']))

,s.tim,s.mat,s.bkts,s.choi,s.cons,s.ign,s.str,s.sup,s.sol,a.noiA,a.noiP,a.supP,a.supT,t.shape,t.tab
10,0.90,0.333,255,14094,12971,0,81,81,Optimal,0,simple,hard,2,"[3, 3, 3, 3]",complete
18,0.10,0.556,63,972,683,0,27,27,Optimal,0,simple,hard,2,"[3, 3, 3]",complete
11,0.34,0.568,255,5346,3791,0,189,189,Optimal,0,simple,hard,4,"[3, 3, 3, 3]",complete
7,240.68,0.601,1330,1330000,368312,0,0,983,Optimal,0,simple,hard,4,"[10, 10, 10]",random
2,2085.81,0.684,1330,330000,66329,0,1000,1000,Optimal,0,simple,hard,2,"[10, 10, 10]",complete
3,2100.55,0.684,1330,330000,66329,0,1000,1000,Optimal,0,simple,hard,4,"[10, 10, 10]",complete
29,1.01,0.704,215,11250,4589,0,125,125,Optimal,0,simple,hard,4,"[5, 5, 5]",complete
28,1.05,0.704,215,11250,4589,0,125,125,Optimal,0,simple,hard,2,"[5, 5, 5]",complete
33,15.20,0.728,215,26875,14234,0,0,145,Optimal,0,simple,hard,4,"[5, 5, 5]",random
19,0.06,0.741,63,243,89,0,54,54,Optimal,0,simple,hard,4,"[3, 3, 3]",complete
